In [1]:
!pip install datasets --quiet
from datasets import load_dataset

In [2]:
dataset = load_dataset("oscar", "unshuffled_deduplicated_ar", split="train[:150000]")

print(len(dataset))

for example in dataset.select(range(3)):
    print(example['text'])

150000
مرحبا بك عزيز الزائر نتمنى لك أوقاتاً سعيدة معنا وأن نزداد شرفا بخدمتك ولا تنسى التسجيل معنا لتستفيد بكل جديد
أهلا وسهلا بك زائرنا الكريم, أنت لم تقم بتسجيل الدخول بعد! يشرفنا أن تقوم بالدخول أو التسجيل إذا رغبت بالمشاركة في المنتدى
نرحب بكل الزائرين ... ونتمى لكم قضاء وقت ممتع معنا ... يملأه الحب والود ... والاستفادة المتبادلة بيننا علميا وعمليا .....
يسعدنا تسجيلكم معنا ومشاركتنا .... وشعارنا دوما (( نحب الخير لكل الناس .. مهما اختلفت الألوان والديانات والأجناس ))
لي أربع شقيقات، أنا أكثرهن غنى، لكن لا أدري لماذا يأتي أقاربي لزيارة أخواتي بكثرة،وحينما يأتي موعد زيارتي لا يأتي سوى القليل،فهم يزورون أخواتي الأربع كل يوم،أما
أنا أكثر أخواتي عطاءً لمن يأتيني.... لا أتهم أخواتي بالتقصير أبداً، ولكن الكل يعرف أني أكثرهن عطاءً...
كثيرون ينصحون أقاربي بأن يأتوني، فلدي خير كثير وأعطي بكرم من يأتيني ومع ذلك يبتعدون عني، فلا حياة لمن تنادي ..
اختر منتدى||--قناة عجباوي التلفزيونية التسويقية| |--البرامج التلفزيونية لقناة عجباوي التسويقية| |--تعرف على عجباوي (سيرة ذاتية - سابقة أعمال)| |--أ

In [3]:
import re
import random

def clean_and_extract_examples(text, max_len=5):
    # Clean
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'[\u064B-\u065F]', '', text)
    text = text.strip()
    
    words = text.split()
    if len(words) < 2:
        return None

    n = random.randint(1, min(max_len, len(words)-1))
    input_seq = words[:n]
    target_word = words[n]

    return (" ".join(input_seq), target_word)


In [4]:
examples = []
for entry in dataset.select(range(150000)):  # Lower for speed
    result = clean_and_extract_examples(entry['text'])
    if result:
        examples.append(result)

for inp, target in examples[:5]:
    print(f"Input: {inp} → Target: {target}")


Input: مرحبا بك → Target: عزيز
Input: ش سوره الفاتحه → Target: سوره
Input: الزلابيه من الحلويات الشهيه و → Target: الذيذه
Input: اعلنت منظمه اطباء بلا حدود → Target: غير
Input: ملاحظه بالنسبه لزوار → Target: انتو


In [5]:
from collections import Counter

# Flatten all words from inputs and targets
all_words = []
for inp, target in examples:
    all_words.extend(inp.split())
    all_words.append(target)

# Count and build vocab
word_freq = Counter(all_words)
min_freq = 5  # remove rare words
vocab = [word for word, freq in word_freq.items() if freq >= min_freq]

# Add special tokens
vocab = ['<PAD>', '<UNK>'] + sorted(vocab)
word2id = {word: idx for idx, word in enumerate(vocab)}
id2word = {idx: word for word, idx in word2id.items()}

vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 13974


In [6]:
def encode_words(words, word2id, max_len):
    tokens = [word2id.get(w, word2id['<UNK>']) for w in words.split()]
    # Padding
    while len(tokens) < max_len:
        tokens.insert(0, word2id['<PAD>'])
    return tokens[-max_len:]  # Truncate from left if too long

def encode_target(word, word2id):
    return word2id.get(word, word2id['<UNK>'])


In [7]:
max_input_len = 5

X = []
y = []

for inp, target in examples:
    X.append(encode_words(inp, word2id, max_input_len))
    y.append(encode_target(target, word2id))


In [8]:
print("Encoded X[0]:", X[0])
print("Target y[0]:", y[0])
print("Decoded X[0]:", [id2word[i] for i in X[0]])
print("Decoded y[0]:", id2word[y[0]])

Encoded X[0]: [0, 0, 0, 11512, 5999]
Target y[0]: 9425
Decoded X[0]: ['<PAD>', '<PAD>', '<PAD>', 'مرحبا', 'بك']
Decoded y[0]: عزيز


In [9]:
import numpy as np
from tensorflow.keras.utils import to_categorical

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)


2025-05-12 23:51:51.303603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747093911.540744     154 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747093911.609134     154 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam


model = Sequential([
    Embedding(vocab_size, 128, input_length=max_input_len),
    LSTM(256, return_sequences=True),
    LSTM(128),
    Dropout(0.2),
    Dense(vocab_size, activation='softmax')
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-05-12 23:51:59.830667: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(X, y, batch_size=64, epochs=30

Epoch 1/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 142s 58ms/step - accuracy: 0.1865 - loss: 7.5839
Epoch 2/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 138s 59ms/step - accuracy: 0.1866 - loss: 7.0191
Epoch 3/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 131s 56ms/step - accuracy: 0.1998 - loss: 6.6736
Epoch 4/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 142s 61ms/step - accuracy: 0.2158 - loss: 6.3244
Epoch 5/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 132s 57ms/step - accuracy: 0.2330 - loss: 5.9667
Epoch 6/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 133s 57ms/step - accuracy: 0.2506 - loss: 5.6203
Epoch 7/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 132s 56ms/step - accuracy: 0.2681 - loss: 5.2826
Epoch 8/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 132s 56ms/step - accuracy: 0.2823 - loss: 4.9726
Epoch 9/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 144s 61ms/step - accuracy: 0.2971 - loss: 4.6784
Epoch 10/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 130s 55ms/step - accuracy: 0.3142 - loss: 4.4023
Epoch 11/30
2337/2337 ━━━━━━━━━━━━━━━━━━━━ 130s 56ms/step - accuracy: 0.3279 - loss: 4.16

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_next_words(text, model, word2id, id2word, max_len=5, top_k=3):
    # Preprocess the text
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'[\u064B-\u065F]', '', text)
    text = text.strip()

    words = text.split()
    if not words:
        return ["⚠️ Please enter at least one Arabic word."]

    # Convert words to tokens
    tokens = [word2id.get(w, word2id['<UNK>']) for w in words]
    while len(tokens) < max_len:
        tokens.insert(0, word2id['<PAD>'])
    tokens = tokens[-max_len:]

    # Prepare input array and get predictions
    input_array = np.array(tokens).reshape(1, max_len)
    pred_probs = model.predict(input_array, verbose=0)[0]

    # Exclude <UNK> and get the highest probability word
    unk_index = word2id["<UNK>"]
    pred_probs[unk_index] = -np.inf
    max_index = np.argmax(pred_probs)
    next_word = id2word[max_index]

    return [next_word]

In [24]:
!pip install gradio

In [32]:
import gradio as gr

def live_autocomplete(user_input):
    if user_input.endswith(" "):
        clean_input = user_input.strip()
        suggestions = predict_next_words(clean_input, model, word2id, id2word)
        return " | ".join(suggestions)
    return "يجب أن تكتب كلمة أولاً ثم تضغط مسافة"

interface = gr.Interface(
    fn=live_autocomplete,
    inputs=gr.Textbox(lines=1, label="اكتب بالعربية", placeholder="مثال: السلام "),
    outputs=gr.Text(label="الكلمات المقترحة التالية"),
    live=True,
    title="نظام اقتراح الكلمات بالعربية",
    description="اكتب ثم اضغط مسافة لرؤية الكلمات المقترحة التالية."
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7869
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://c049776cb0a72779af.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [33]:
model.save('/kaggle/working/my_model.h5')